In [1]:
import sys
import numpy as np
sys.path.append('..//')

In [12]:
from radio.batchflow import Dataset, Pipeline, DatasetIndex, L, F
from radio.batchflow import NumpySampler as NS, Sampler, ConstantSampler as CS
from radio import CTImagesAugmentedBatch  # augmentation batch

In [4]:
dsix = DatasetIndex(np.arange(100))
ds = Dataset(index=dsix, batch_class=CTImagesAugmentedBatch)

### Augmentation samplers

In [5]:
sh = np.array([32, 64, 64])
delta = np.array([16, 24, 24])
pos = (NS('n', dim=3) * sh / 4 + sh / 2 + delta // 2).truncate(sh - 1, (0, 0, 0))
sz = NS('choice', a=np.arange(7, 13)) * np.array([1, 2, 2])
noise = NS('n') * 0.1
sp = (1 + 0.3 * NS('n', dim=3)).truncate(low=(0.5, 0.5, 0.5))

### Augmentation pipeline

In [6]:
pipeline = (Pipeline()
            .init_with_ones(sh + delta)
            .cutout(positions=L(pos.sample, size=F(len)), sizes=L(sz.sample, size=F(len)))
            .apply_noise(noise)
            .rotate(angle=30)
            .unify_spacing(spacing=(L(sp.sample, size=1)), shape=sh, mode='constant')
           )

In [7]:
bch = (ds >> pipeline).next_batch(3)

In [9]:
def plot_2d(arry, clim=(-0.3, 1.3)):
    plt.imshow(arry, cmap=plt.cm.gray, clim=clim)

In [10]:
from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
interact(lambda num, slc: plot_2d(bch.get(num, 'images')[slc], clim=(0, 1.3)),
         num=(0, len(bch) - 1, 1), slc=(0, len(bch.get(0, 'images')) - 1, 1))

interactive(children=(IntSlider(value=1, description='num', max=2), IntSlider(value=15, description='slc', max…

<function __main__.<lambda>>